# Insper - Pós Graduação em Ciência de Dados

## Computação para Ciência de Dados

### Projeto Integrador - Label Engineering

# Preparação

In [26]:
# Bibliotecas
import pandas as pd
import numpy as np

# Importando os dados

In [27]:
df = pd.read_csv('cs_bisnode_panel.csv')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287829 entries, 0 to 287828
Data columns (total 48 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   comp_id               287829 non-null  float64
 1   begin                 287829 non-null  object 
 2   end                   287829 non-null  object 
 3   COGS                  18257 non-null   float64
 4   amort                 279789 non-null  float64
 5   curr_assets           287698 non-null  float64
 6   curr_liab             287698 non-null  float64
 7   extra_exp             269300 non-null  float64
 8   extra_inc             269300 non-null  float64
 9   extra_profit_loss     270626 non-null  float64
 10  finished_prod         17485 non-null   float64
 11  fixed_assets          287698 non-null  float64
 12  inc_bef_tax           280392 non-null  float64
 13  intang_assets         287689 non-null  float64
 14  inventories           287698 non-null  float64
 15  

# Filtro inicial

Dropando as colunas `COGS`, `finished_prod`, `net_dom_sales`, `net_exp_sales`, `wages`, `D`. 

In [29]:
filt_df = df.drop(['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D'], axis=1)

Removendo o ano de `2016`

In [30]:
filt_df = filt_df[filt_df.year != 2016]

# Label Engineering

Defina o índice do dataframe com as colunas `year` e `comp_id`

In [31]:
year = filt_df['year'].unique()
comp_id = filt_df['comp_id'].unique()

Utilize o método `MultiIndex.from_product` para complementar índices faltantes para cada `comp_id`

In [32]:
filt_df = filt_df.set_index(['year', 'comp_id'])

Utilize o método `DataFrame.reindex` para criar as linhas nulas nos anos faltantes de cada `comp_id`

In [33]:
multi_index = pd.MultiIndex.from_product([year, comp_id],
                           names=['year', 'comp_id'])

filt_df = filt_df.reindex(multi_index).reset_index()


Criar uma coluna `status_alive` que indica 1 para quando uma empresa está em operação em um determinado ano, 0 caso contrário

In [34]:
filt_df['sales']= filt_df.sales.fillna(0)
filt_df["status_alive"] = ((filt_df["sales"] > 0) & (~filt_df["sales"].isna())).astype(int)

Criar uma coluna `default` que indica 1 caso uma empres faliu em dois anos respectivo a cada ano, 0 caso contrário

In [35]:
filt_df = filt_df.sort_values(['comp_id','year'], ascending=[0,1])
filt_df['default'] = ((filt_df['status_alive'] == 1) & (filt_df.groupby("comp_id")["status_alive"].shift(-2) == 0)
                        & (filt_df.groupby("comp_id")["status_alive"].shift(-1) == 1)).astype(int)

# Filtrando e tratando observações

Vamos retirar as observações com vendas negativas

In [36]:
neg_compid = (df[df.sales < 0])['comp_id'].unique()
filt_df = filt_df[~filt_df['comp_id'].isin(neg_compid)]

Filtrar por vendas entre 1.000 e 10MM de Euros.

In [37]:
filt_df = filt_df.query("sales < 10000000 & sales > 1000")

Aplicar o log na coluna sales.

In [38]:
filt_df['log_sales'] = np.where(filt_df.sales > 0 , np.log(filt_df.sales), 0)

Remover observações com status_alive == 0 em 2012

In [39]:
filt_df = filt_df.query('year == 2012 & status_alive == 1')

Remover observações anteriores à 2012

In [40]:
filt_df = filt_df.query('year == 2012')

### Novas colunas de idade

Criar coluna idade da empresa em 2012

In [41]:
filt_df['age'] = (2012 - filt_df.founded_year)

Criar coluna idade média dos CEOs da empresa em 2012

In [42]:
filt_df['ceo_avgage'] = (2012 - filt_df.birth_year)

### Novas colunas financeiras

Criar coluna Liquidez Corrente

In [43]:
filt_df['curr_liq'] = filt_df["curr_assets"]/filt_df["curr_liab"]

Criar coluna Liquidez Seca

In [44]:
filt_df['curr_liq'] = filt_df["liq_assets"]/filt_df["curr_liab"]

Criar Margem Líquida

In [45]:
filt_df['net_profit'] = filt_df["profit_loss_year"]/filt_df["sales"]

Criando grupos de colunas

In [46]:
neg_group = ["amort", "curr_assets","curr_liab","fixed_assets","intang_assets", "inventories","liq_assets","tang_assets","share_eq","subscribed_cap",
            "extra_exp", "extra_inc", "extra_profit_loss", "inc_bef_tax", "material_exp", "personnel_exp", "profit_loss_year", "age"]

list_neg = []

for column in neg_group:
    nunique = filt_df[filt_df[column] < 0]['comp_id'].nunique()
    list_nunique = filt_df[filt_df[column] < 0]['comp_id'].unique()
    list_neg.append([column, nunique, list_nunique])

pd.DataFrame(list_neg, columns = ["column", "nunique", "compid_list"])

,column,nunique,compid_list
0,amort,6,"[244979122176.0, 240819044352.0, 225678131200...."
1,curr_assets,11,"[441549553664.0, 323038543872.0, 292542283776...."
2,curr_liab,11,"[419201286144.0, 390831079424.0, 180938506240...."
3,fixed_assets,0,[]
4,intang_assets,2,"[87559397376.0, 37105459200.0]"
5,inventories,4,"[323038543872.0, 104754192384.0, 75834507264.0..."
6,liq_assets,35,"[441549553664.0, 412312862720.0, 406537830400...."
7,tang_assets,0,[]
8,share_eq,8086,"[463829172224.0, 463507849216.0, 463351349248...."
9,subscribed_cap,1,[419277373440.0]


In [52]:
neg_group = ["amort", "curr_assets","curr_liab","fixed_assets","intang_assets", "inventories","liq_assets","tang_assets","share_eq","subscribed_cap",
            "extra_exp", "extra_inc", "extra_profit_loss", "inc_bef_tax", "material_exp", "personnel_exp", "profit_loss_year", "age"]

list_na = []

for column in neg_group:
    nunique = filt_df[filt_df[column].isna()]['comp_id'].nunique()
    list_nunique = filt_df[filt_df[column].isna()]['comp_id'].unique()
    list_na.append([column, nunique, list_nunique])

pd.DataFrame(list_na, columns = ["column", "nunique", "compid_list"])

,column,nunique,compid_list
0,amort,40,"[451886055424.0, 435282083840.0, 413635313664...."
1,curr_assets,10,"[370157748224.0, 329876963328.0, 271439691776...."
2,curr_liab,10,"[370157748224.0, 329876963328.0, 271439691776...."
3,fixed_assets,10,"[370157748224.0, 329876963328.0, 271439691776...."
4,intang_assets,10,"[370157748224.0, 329876963328.0, 271439691776...."
5,inventories,10,"[370157748224.0, 329876963328.0, 271439691776...."
6,liq_assets,10,"[370157748224.0, 329876963328.0, 271439691776...."
7,tang_assets,10,"[370157748224.0, 329876963328.0, 271439691776...."
8,share_eq,10,"[370157748224.0, 329876963328.0, 271439691776...."
9,subscribed_cap,10,"[370157748224.0, 329876963328.0, 271439691776...."
